# Summary

----

# Imports

In [1]:
%run imports.ipynb

/home/kimlab1/strokach/anaconda/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/home/kimlab1/strokach/anaconda/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
INFO - root - Initialized logger
ERROR - root - Initialized logger


2016-06-06 15:38:00.183658


In [2]:
%run mysqld.ipynb

INFO - root - Initialized logger
ERROR - root - Initialized logger
DEBUG - datapkg.daemons - 'basedir': /home/kimlab1/strokach/anaconda
INFO - root - MySQL database already running...


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2016-06-06 15:38:00.368735


In [3]:
NOTEBOOK_NAME = 'load_data'
NOTEBOOK_NAME

'load_data'

In [4]:
CONNECTION_STRING = mysqld.get_connection_string(SCHEMA_NAME)
CONNECTION_STRING

'mysql://root:@localhost:8301/dbnsfp?unix_socket=/tmp/strokach/dbnsfp/mysql.sock'

In [5]:
reload(datapkg)

<module 'datapkg' from '/home/kimlab1/strokach/working/datapkg/datapkg/__init__.py'>

In [6]:
db = datapkg.FileToMySQL(CONNECTION_STRING, NOTEBOOK_NAME, None, echo=False, db_engine='MyISAM')

# Extract

In [7]:
!ls ../downloads

dbNSFPv3.2a.zip


In [8]:
PWD = os.getcwd()
STG_SERVER_IP = os.environ['STG_SERVER_IP']

Note: do this on the storage server so as not to overload the network:

In [9]:
!ls -lh $NOTEBOOK_NAME

total 90G
-rw------- 1 strokach kimlab  78K Mar 31 00:50 dbNSFP3.2a.readme.txt
-rw------- 1 strokach kimlab 9.6G Jun  6 14:52 dbNSFP3.2a_variant.chr1
-rw------- 1 strokach kimlab 3.5G Mar 18 22:30 dbNSFP3.2a_variant.chr10
-rw------- 1 strokach kimlab 5.3G Mar 18 22:37 dbNSFP3.2a_variant.chr11
-rw------- 1 strokach kimlab 4.8G Mar 18 22:43 dbNSFP3.2a_variant.chr12
-rw------- 1 strokach kimlab 1.6G Mar 18 22:45 dbNSFP3.2a_variant.chr13
-rw------- 1 strokach kimlab 2.9G Mar 18 22:49 dbNSFP3.2a_variant.chr14
-rw------- 1 strokach kimlab 3.1G Mar 18 22:53 dbNSFP3.2a_variant.chr15
-rw------- 1 strokach kimlab 3.8G Mar 18 22:58 dbNSFP3.2a_variant.chr16
-rw------- 1 strokach kimlab 5.2G Mar 18 23:05 dbNSFP3.2a_variant.chr17
-rw------- 1 strokach kimlab 1.4G Mar 18 23:07 dbNSFP3.2a_variant.chr18
-rw------- 1 strokach kimlab 5.7G Mar 18 23:15 dbNSFP3.2a_variant.chr19
-rw------- 1 strokach kimlab 6.8G Mar 18 23:24 dbNSFP3.2a_variant.chr2
-rw------- 1 strokach kimlab 2.1G Mar 18 23:27 dbNSFP3.2a_v

### gene

### gene_complete

### dbNSFP3.2a_variant.chr1

In [10]:
variant_chr1 = pd.read_csv(
    op.join(NOTEBOOK_NAME, 'dbNSFP3.2a_variant.chr1'), sep='\t', nrows=100, na_values=['.', '\\N'])
display(variant_chr1.head(2))

,#chr,pos(1-based),ref,alt,aaref,aaalt,rs_dbSNP146,hg19_chr,hg19_pos(1-based),hg18_chr,hg18_pos(1-based),genename,cds_strand,refcodon,codonpos,codon_degeneracy,Ancestral_allele,AltaiNeandertal,Denisova,Ensembl_geneid,Ensembl_transcriptid,Ensembl_proteinid,aapos,SIFT_score,SIFT_converted_rankscore,SIFT_pred,Uniprot_acc_Polyphen2,Uniprot_id_Polyphen2,Uniprot_aapos_Polyphen2,Polyphen2_HDIV_score,Polyphen2_HDIV_rankscore,Polyphen2_HDIV_pred,Polyphen2_HVAR_score,Polyphen2_HVAR_rankscore,Polyphen2_HVAR_pred,LRT_score,LRT_converted_rankscore,LRT_pred,LRT_Omega,MutationTaster_score,MutationTaster_converted_rankscore,MutationTaster_pred,MutationTaster_model,MutationTaster_AAE,MutationAssessor_UniprotID,MutationAssessor_variant,MutationAssessor_score,MutationAssessor_score_rankscore,MutationAssessor_pred,FATHMM_score,FATHMM_converted_rankscore,FATHMM_pred,PROVEAN_score,PROVEAN_converted_rankscore,PROVEAN_pred,Transcript_id_VEST3,Transcript_var_VEST3,VEST3_score,VEST3_rankscore,MetaSVM_score,MetaSVM_rankscore,MetaSVM_pred,MetaLR_score,MetaLR_rankscore,MetaLR_pred,Reliability_index,CADD_raw,CADD_raw_rankscore,CADD_phred,DANN_score,DANN_rankscore,fathmm-MKL_coding_score,fathmm-MKL_coding_rankscore,fathmm-MKL_coding_pred,fathmm-MKL_coding_group,Eigen-raw,Eigen-phred,Eigen-raw_rankscore,Eigen-PC-raw,Eigen-PC-raw_rankscore,GenoCanyon_score,GenoCanyon_score_rankscore,integrated_fitCons_score,integrated_fitCons_score_rankscore,integrated_confidence_value,GM12878_fitCons_score,GM12878_fitCons_score_rankscore,GM12878_confidence_value,H1-hESC_fitCons_score,H1-hESC_fitCons_score_rankscore,H1-hESC_confidence_value,HUVEC_fitCons_score,HUVEC_fitCons_score_rankscore,HUVEC_confidence_value,GERP++_NR,GERP++_RS,GERP++_RS_rankscore,phyloP100way_vertebrate,phyloP100way_vertebrate_rankscore,phyloP20way_mammalian,phyloP20way_mammalian_rankscore,phastCons100way_vertebrate,phastCons100way_vertebrate_rankscore,phastCons20way_mammalian,phastCons20way_mammalian_rankscore,SiPhy_29way_pi,SiPhy_29way_logOdds,SiPhy_29way_logOdds_rankscore,1000Gp3_AC,1000Gp3_AF,1000Gp3_AFR_AC,1000Gp3_AFR_AF,1000Gp3_EUR_AC,1000Gp3_EUR_AF,1000Gp3_AMR_AC,1000Gp3_AMR_AF,1000Gp3_EAS_AC,1000Gp3_EAS_AF,1000Gp3_SAS_AC,1000Gp3_SAS_AF,TWINSUK_AC,TWINSUK_AF,ALSPAC_AC,ALSPAC_AF,ESP6500_AA_AC,ESP6500_AA_AF,ESP6500_EA_AC,ESP6500_EA_AF,ExAC_AC,ExAC_AF,ExAC_Adj_AC,ExAC_Adj_AF,ExAC_AFR_AC,ExAC_AFR_AF,ExAC_AMR_AC,ExAC_AMR_AF,ExAC_EAS_AC,ExAC_EAS_AF,ExAC_FIN_AC,ExAC_FIN_AF,ExAC_NFE_AC,ExAC_NFE_AF,ExAC_SAS_AC,ExAC_SAS_AF,ExAC_nonTCGA_AC,ExAC_nonTCGA_AF,ExAC_nonTCGA_Adj_AC,ExAC_nonTCGA_Adj_AF,ExAC_nonTCGA_AFR_AC,ExAC_nonTCGA_AFR_AF,ExAC_nonTCGA_AMR_AC,ExAC_nonTCGA_AMR_AF,ExAC_nonTCGA_EAS_AC,ExAC_nonTCGA_EAS_AF,ExAC_nonTCGA_FIN_AC,ExAC_nonTCGA_FIN_AF,ExAC_nonTCGA_NFE_AC,ExAC_nonTCGA_NFE_AF,ExAC_nonTCGA_SAS_AC,ExAC_nonTCGA_SAS_AF,ExAC_nonpsych_AC,ExAC_nonpsych_AF,ExAC_nonpsych_Adj_AC,ExAC_nonpsych_Adj_AF,ExAC_nonpsych_AFR_AC,ExAC_nonpsych_AFR_AF,ExAC_nonpsych_AMR_AC,ExAC_nonpsych_AMR_AF,ExAC_nonpsych_EAS_AC,ExAC_nonpsych_EAS_AF,ExAC_nonpsych_FIN_AC,ExAC_nonpsych_FIN_AF,ExAC_nonpsych_NFE_AC,ExAC_nonpsych_NFE_AF,ExAC_nonpsych_SAS_AC,ExAC_nonpsych_SAS_AF,clinvar_rs,clinvar_clnsig,clinvar_trait,clinvar_golden_stars,Interpro_domain,GTEx_V6_gene,GTEx_V6_tissue
0,1,69091,A,C,M,L,NaN,1,69091,1,58954,OR4F5,+,ATG,1,0,NaN,./.,./.,ENSG00000186092,ENST00000335137,ENSP00000334393,1,0.13,0.26702,T,Q8NH21,OR4F5_HUMAN,1.0,0.0,0.02634,B,0.0,0.01257,B,0.589091,0.05577,N,1.33974,NaN,NaN,NaN,NaN,NaN,OR4F5_HUMAN,M1L,NaN,NaN,NaN,6.76,0.00529,T,-0.38,0.13435,N,NM_001005484.1,M1L,0.120,0.13350,-0.9577,0.39629,T,0.0009,0.00318,T,8.0,-1.436194,0.01840,0.003,0.443783,0.03370,0.02063,0.06083,N,AEFI,-1.394131,0.119256,0.02697,-1.535705,0.02036,0.000022,0.0355,0.487112,0.13308,0,0.573888,0.26071,0,0.573888,0.22998,0,0.564101,0.26208,0,2.31,-4.63,0.03101,-0.055,0.11668,-1.983,0.00506,0.0,0.06329,0.0,0.01567,0.2547:0.0:0.5282:0.2171,3.5592,0.07372,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [11]:
variant_chr1.columns = datapkg.format_columns(variant_chr1.columns)
display(variant_chr1.head(2))

,chr,pos_1based,ref,alt,aaref,aaalt,rs_dbsnp146,hg19_chr,hg19_pos_1based,hg18_chr,hg18_pos_1based,genename,cds_strand,refcodon,codonpos,codon_degeneracy,ancestral_allele,altai_neandertal,denisova,ensembl_geneid,ensembl_transcriptid,ensembl_proteinid,aapos,sift_score,sift_converted_rankscore,sift_pred,uniprot_acc_polyphen2,uniprot_id_polyphen2,uniprot_aapos_polyphen2,polyphen2_hdiv_score,polyphen2_hdiv_rankscore,polyphen2_hdiv_pred,polyphen2_hvar_score,polyphen2_hvar_rankscore,polyphen2_hvar_pred,lrt_score,lrt_converted_rankscore,lrt_pred,lrt_omega,mutationtaster_score,mutationtaster_converted_rankscore,mutationtaster_pred,mutationtaster_model,mutationtaster_aae,mutationassessor_uniprotid,mutationassessor_variant,mutationassessor_score,mutationassessor_score_rankscore,mutationassessor_pred,fathmm_score,fathmm_converted_rankscore,fathmm_pred,provean_score,provean_converted_rankscore,provean_pred,transcript_id_vest3,transcript_var_vest3,vest3_score,vest3_rankscore,metasvm_score,metasvm_rankscore,metasvm_pred,metalr_score,metalr_rankscore,metalr_pred,reliability_index,cadd_raw,cadd_raw_rankscore,cadd_phred,dann_score,dann_rankscore,fathmmmkl_coding_score,fathmmmkl_coding_rankscore,fathmmmkl_coding_pred,fathmmmkl_coding_group,eigenraw,eigenphred,eigenraw_rankscore,eigenpcraw,eigenpcraw_rankscore,genocanyon_score,genocanyon_score_rankscore,integrated_fitcons_score,integrated_fitcons_score_rankscore,integrated_confidence_value,gm12878_fitcons_score,gm12878_fitcons_score_rankscore,gm12878_confidence_value,h1hesc_fitcons_score,h1hesc_fitcons_score_rankscore,h1hesc_confidence_value,huvec_fitcons_score,huvec_fitcons_score_rankscore,huvec_confidence_value,gerp_nr,gerp_rs,gerp_rs_rankscore,phylop100way_vertebrate,phylop100way_vertebrate_rankscore,phylop20way_mammalian,phylop20way_mammalian_rankscore,phastcons100way_vertebrate,phastcons100way_vertebrate_rankscore,phastcons20way_mammalian,phastcons20way_mammalian_rankscore,siphy_29way_pi,siphy_29way_logodds,siphy_29way_logodds_rankscore,gp3_ac,gp3_af,gp3_afr_ac,gp3_afr_af,gp3_eur_ac,gp3_eur_af,gp3_amr_ac,gp3_amr_af,gp3_eas_ac,gp3_eas_af,gp3_sas_ac,gp3_sas_af,twinsuk_ac,twinsuk_af,alspac_ac,alspac_af,esp6500_aa_ac,esp6500_aa_af,esp6500_ea_ac,esp6500_ea_af,exac_ac,exac_af,exac_adj_ac,exac_adj_af,exac_afr_ac,exac_afr_af,exac_amr_ac,exac_amr_af,exac_eas_ac,exac_eas_af,exac_fin_ac,exac_fin_af,exac_nfe_ac,exac_nfe_af,exac_sas_ac,exac_sas_af,exac_nontcga_ac,exac_nontcga_af,exac_nontcga_adj_ac,exac_nontcga_adj_af,exac_nontcga_afr_ac,exac_nontcga_afr_af,exac_nontcga_amr_ac,exac_nontcga_amr_af,exac_nontcga_eas_ac,exac_nontcga_eas_af,exac_nontcga_fin_ac,exac_nontcga_fin_af,exac_nontcga_nfe_ac,exac_nontcga_nfe_af,exac_nontcga_sas_ac,exac_nontcga_sas_af,exac_nonpsych_ac,exac_nonpsych_af,exac_nonpsych_adj_ac,exac_nonpsych_adj_af,exac_nonpsych_afr_ac,exac_nonpsych_afr_af,exac_nonpsych_amr_ac,exac_nonpsych_amr_af,exac_nonpsych_eas_ac,exac_nonpsych_eas_af,exac_nonpsych_fin_ac,exac_nonpsych_fin_af,exac_nonpsych_nfe_ac,exac_nonpsych_nfe_af,exac_nonpsych_sas_ac,exac_nonpsych_sas_af,clinvar_rs,clinvar_clnsig,clinvar_trait,clinvar_golden_stars,interpro_domain,gtex_v6_gene,gtex_v6_tissue
0,1,69091,A,C,M,L,NaN,1,69091,1,58954,OR4F5,+,ATG,1,0,NaN,./.,./.,ENSG00000186092,ENST00000335137,ENSP00000334393,1,0.13,0.26702,T,Q8NH21,OR4F5_HUMAN,1.0,0.0,0.02634,B,0.0,0.01257,B,0.589091,0.05577,N,1.33974,NaN,NaN,NaN,NaN,NaN,OR4F5_HUMAN,M1L,NaN,NaN,NaN,6.76,0.00529,T,-0.38,0.13435,N,NM_001005484.1,M1L,0.120,0.13350,-0.9577,0.39629,T,0.0009,0.00318,T,8.0,-1.436194,0.01840,0.003,0.443783,0.03370,0.02063,0.06083,N,AEFI,-1.394131,0.119256,0.02697,-1.535705,0.02036,0.000022,0.0355,0.487112,0.13308,0,0.573888,0.26071,0,0.573888,0.22998,0,0.564101,0.26208,0,2.31,-4.63,0.03101,-0.055,0.11668,-1.983,0.00506,0.0,0.06329,0.0,0.01567,0.2547:0.0:0.5282:0.2171,3.5592,0.07372,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [12]:
def load_table_to_db(suffix, dtypes=None, auto_increment=1):
    file = op.abspath(op.join(NOTEBOOK_NAME, 'dbNSFP3.2a_variant.chr{}'.format(suffix)))
    tablename = 'variant_chr{}'.format(suffix)
    
    # Replace ';' to ',' so that FIND_IN_SET works
    datapkg.run_command("sed -i 's/;/,/g' '{}'".format(file))
    
    # Load file to database
    t = db.import_table(
        filename=file,
        tablename=tablename,
        # na_values=['\\N'],
        na_values=['.', '\\N'],
        dtypes=dtypes,
        extra_dtypes={'chr': sa.VARCHAR(2)}
    )
    
    # Add an autoincrement 'id' column
    max_id = db.add_id_column(t.name, auto_increment=auto_increment)
    
    # Create indices
    db.create_indexes(
        t.name,
        index_commands=[
            [('chr', 'pos_1based', 'ref', 'alt'), False],
            [('chr', 'pos_1based', 'aaref', 'aaalt'), False],
            [('rs_dbsnp146', ), False],
            [('aaref', 'aaalt', ), False],
            [('ensembl_geneid', 'aaref', 'aaalt', ), False],
            [('genename', ), False],
            [('uniprot_id_polyphen2', 'uniprot_aapos_polyphen2', ), False],
            [('mutationassessor_uniprotid ', 'mutationassessor_variant', ), False],
        ],
    )
    return t, max_id

In [ ]:
chr1_t, chr1_max_id = load_table_to_db(1)

DEBUG - datapkg._helper - sed -i 's/;/,/g' '/kimlab1/database_data/biodb/recipes/dbnsfp/notebooks/load_data/dbNSFP3.2a_variant.chr1'
DEBUG - datapkg._helper - Running locally
DEBUG - datapkg._helper - Command ran successfully:

DEBUG - datapkg._helper - File '/kimlab1/database_data/biodb/recipes/dbnsfp/notebooks/load_data/dbNSFP3.2a_variant' is not compressed...
DEBUG - datapkg._helper - sed -i -e 's/\t\.\t/\t\\N\t/g' -e 's/\t\.\t/\t\\N\t/g' -e 's/^\.\t/\\N\t/g' -e 's/\t\.$/\t\\N/g' -e 's/\t\.\r$/\t\\N/g'  /kimlab1/database_data/biodb/recipes/dbnsfp/notebooks/load_data/dbNSFP3.2a_variant.chr1
DEBUG - datapkg._helper - Running locally
DEBUG - datapkg._helper - Command ran successfully:

DEBUG - datapkg._df_helper - get_file_dtypes(/kimlab1/database_data/biodb/recipes/dbnsfp/notebooks/load_data/dbNSFP3.2a_variant.chr1, 100000, 100, {'sep': '\t', 'na_values': ['.', '\\N']})
DEBUG - datapkg._helper - wc -l '/kimlab1/database_data/biodb/recipes/dbnsfp/notebooks/load_data/dbNSFP3.2a_variant.

In [ ]:
max_id = chr1_max_id
for suffix in list(list(range(1, 23)) + ['M', 'X', 'Y']):
    print(suffix)
    t, max_id = load_table_to_db(suffix, dtypes=chr1_t.dtypes, auto_increment=max_id+1)

DEBUG - datapkg._helper - sed -i 's/;/,/g' '/kimlab1/database_data/biodb/recipes/dbnsfp/notebooks/load_data/dbNSFP3.2a_variant.chr1'
DEBUG - datapkg._helper - Running locally


In [36]:
columns = ',\n'.join('  `{}` {}'.format(c, chr1_t.dtypes[c].compile()) for c in t.df.columns)

sql_command = """\
  CREATE OR REPLACE TABLE `variant` (
  `id` bigint(20) NOT NULL AUTO_INCREMENT,
{columns},
  PRIMARY KEY (`id`),
  KEY `a` (`chr`,`pos_1based`,`ref`,`alt`),
  KEY `b` (`chr`,`pos_1based`,`aaref`,`aaalt`),
  KEY `c` (`rs_dbsnp146`),
  KEY `d` (`aaref`,`aaalt`),
  KEY `e` (`ensembl_geneid`,`aaref`,`aaalt`),
  KEY `f` (`genename`),
  KEY `g` (`uniprot_id_polyphen2`,`uniprot_aapos_polyphen2`),
  KEY `h` (`mutationassessor_uniprotid`,`mutationassessor_variant`)
) ENGINE=MERGE 
  UNION=(variant_chr1, variant_chr2, variant_chr3, variant_chr4, variant_chr5, variant_chr6, variant_chr7, 
         variant_chr8, variant_chr9, variant_chr10, variant_chr11, variant_chr12, variant_chr13, variant_chr14, 
         variant_chr15, variant_chr16, variant_chr17, variant_chr18, variant_chr19, variant_chr20, variant_chr21, 
         variant_chr22, variant_chrM, variant_chrX, variant_chrY)
  DEFAULT CHARSET=latin1;
""".format(columns=columns)
print(sql_command)

  CREATE OR REPLACE TABLE `variant` (
  `id` bigint(20) NOT NULL AUTO_INCREMENT,
  `chr` VARCHAR(2),
  `pos_1based` INTEGER,
  `ref` VARCHAR(32),
  `alt` VARCHAR(32),
  `aaref` VARCHAR(32),
  `aaalt` VARCHAR(32),
  `rs_dbsnp146` VARCHAR(32),
  `hg19_chr` DOUBLE,
  `hg19_pos_1based` DOUBLE,
  `hg18_chr` DOUBLE,
  `hg18_pos_1based` DOUBLE,
  `genename` VARCHAR(255),
  `cds_strand` VARCHAR(32),
  `refcodon` VARCHAR(255),
  `codonpos` DOUBLE,
  `codon_degeneracy` VARCHAR(255),
  `ancestral_allele` VARCHAR(32),
  `altai_neandertal` VARCHAR(32),
  `denisova` VARCHAR(32),
  `ensembl_geneid` VARCHAR(255),
  `ensembl_transcriptid` MEDIUMTEXT,
  `ensembl_proteinid` MEDIUMTEXT,
  `aapos` VARCHAR(255),
  `sift_score` VARCHAR(255),
  `sift_converted_rankscore` DOUBLE,
  `sift_pred` VARCHAR(255),
  `uniprot_acc_polyphen2` MEDIUMTEXT,
  `uniprot_id_polyphen2` VARCHAR(255),
  `uniprot_aapos_polyphen2` VARCHAR(255),
  `polyphen2_hdiv_score` VARCHAR(255),
  `polyphen2_hdiv_rankscore` DOUBLE,
  `polyphen

In [37]:
db.engine.execute(sql_command)

In [38]:
mysqld.myisampack(SCHEMA_NAME)

AttributeError: 'MySQLDaemon' object has no attribute 'data_dir'